# Jigsaw

- Implement an NxN jigsaw puzzle.
- Design the data structures and explain an algorithm to solve the puzzle.
- You can assume that you have a `fitsWith` method which, when passed two puzzle edges, returns `true` if the two edges belong together.

In [93]:
from enum import Enum


def fits_with(edge1: "Edge", edge2: "Edge", N: int) -> bool:
    if edge1.is_border() or edge2.is_border():
        return False
    
    locations = {edge1.location, edge2.location}
    vertical_fit = locations == {EdgeLocation.TOP, EdgeLocation.BOTTOM}
    horizontal_fit = locations == {EdgeLocation.LEFT, EdgeLocation.RIGHT}

    if not vertical_fit and not horizontal_fit:
        return False
    
    if vertical_fit:
        if edge1.location == EdgeLocation.TOP:
            return edge1.piece_value == edge2.piece_value - N
        else:  # edge1 is BOTTOM
            return edge2.piece_value == edge1.piece_value + N
    else:
        if edge1.location == EdgeLocation.LEFT:
            return edge1.piece_value == edge2.piece_value - 1
        else:  # edge1 is RIGHT
            return edge2.piece_value == edge1.piece_value + 1


class EdgeLocation(Enum):
    TOP = 0
    LEFT = 1
    BOTTOM = 2
    RIGHT = 3


class Edge:
    def __init__(self, piece_value: int, location: EdgeLocation, N: int):
        self.piece_value = piece_value
        self.location = location
        self.N = N
        self.top_row_values = set(range(N))
        self.bottom_row_values = set(range((N - 1) * N, N * N))
        self.left_col_values = set(range(0, N * N, N))
        self.right_col_values = set(range(N - 1, N * N, N))

    def fits_with(self, other: "Edge") -> bool:
        return fits_with(self, other, self.N)
    
    def is_border(self) -> bool:
        if self.location == EdgeLocation.TOP:
            return self.piece_value in self.top_row_values
        elif self.location == EdgeLocation.BOTTOM:
            return self.piece_value in self.bottom_row_values
        elif self.location == EdgeLocation.RIGHT:
            return self.piece_value in self.right_col_values
        elif self.location == EdgeLocation.LEFT:
            return self.piece_value in self.left_col_values

        
    def __repr__(self):
        return f"{self.piece_value}-{self.location.name}"


class Piece:
    def __init__(self, value: int, N: int):
        self.value = value
        self.edges = self.top, self.left, self.bottom, self.right = [Edge(value, loc, N) for loc in EdgeLocation]

    def __repr__(self):
        return f"{self.value:02}"
    
    @property
    def fittable_edges(self):
        return [edge for edge in self.edges if not edge.is_border()]


class Puzzle:
    def __init__(self, pieces: list[int]):
        # ! Docstring: clarify that pieces should be set by row, as a numpy 2D-array
        self.N = int(len(pieces)**(1/2))  # Assumes NxN Puzzle
        self.pieces = {Piece(value, self.N) for value in pieces}
        self.unplaced_pieces = set(self.pieces)
        self.placed_pieces = set()
        self.frame = np.empty((self.N, self.N), dtype=object)
        self.frame[:, :] = None
        self.frame = self.frame.tolist()
        
    def place_piece(self, piece: Piece, i: int, j: int):
        if self.frame[i][j] is not None:
            self.unplaced_pieces = self.frame[i][j]
            self.frame[i][j] = None

        self.frame[i][j] = piece
        self.unplaced_pieces.remove(piece)
        self.placed_pieces.add(piece)

    def solve(self):
        # Prime the puzzle placing the top left border
        for piece in self.unplaced_pieces:
            if piece.left.is_border() and piece.top.is_border():
                self.place_piece(piece, 0, 0)
                display(self)

        # Iteratively find the fits of the pieces that have been put, and put those
        while self.unplaced_pieces:
            # TODO COntinue here

    def __repr__(self):
        return (
            f"{len(self.unplaced_pieces)} unplaced pieces: " +
            ", ".join(str(row) for row in self.unplaced_pieces) + "\n" +
            "Frame: " + self.frame.__repr__()
        )

    def __getitem__(self, ij: tuple[int, int]):
        i, j = ij
        return self.pieces[i][j]

    
#####
import numpy as np
import random


pieces = list(range(25))
random.shuffle(pieces)
puzzle = Puzzle(pieces)
puzzle.solve()

24 unplaced pieces: 06, 02, 08, 13, 17, 09, 19, 10, 15, 03, 24, 01, 11, 04, 23, 16, 22, 20, 18, 05, 07, 14, 21, 12
Frame: [[00, None, None, None, None], [None, None, None, None, None], [None, None, None, None, None], [None, None, None, None, None], [None, None, None, None, None]]

In [54]:
p1 = puzzle[0, 0]
p2 = puzzle[1, 1]

p1, p2

(21, 22)

In [55]:
p1.right, p2.left

(21-RIGHT, 22-LEFT)

In [56]:
p1.right.fits_with(p2.left)

True